In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import os
from pathlib import Path
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Configuration

batch_size = 64
epochs = 2000
latent_dim = 1000

In [ ]:
def prepare_data(timeseries_data, n_rows):
    X, y =[],[]
    for i in range(len(timeseries_data)):
        end_ix = i + n_rows
        if end_ix > len(timeseries_data)-1:
            break
        seq_x, seq_y = ''.join(timeseries_data[i:end_ix]), timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return X, y

In [ ]:
best_accuracies = {}


df = pd.read_csv("/kaggle/input/dummy-genome-for-work/Corona Virus Genome Sequences time sorted_v1.csv")

# remove \n and \r
df['GenomeSequence'] = df['GenomeSequence'].apply(lambda x: ''.join(x.split('\n')[1:]))
df['genome_sequence'] = df['GenomeSequence']
df['genome_sequence'] = df['genome_sequence'].str.replace('\r', '')
df['genome_sequence'] = df['genome_sequence'].str.replace('\n', '')
#df['genomes'] = df['genomes'].str.replace('-', '')
df['genome_sequence'] = df['genome_sequence'].str.replace('K', '')
# add . after each GenomeSequence
#df['GenomeSequence'] = df['GenomeSequence'] + '.'


"""
df = pd.DataFrame({'genome_sequence': [
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG',
    'AATCGTATCGCATCG'
]})
"""

sequence_cut_length = 300
#ending_sequence = df['genome_sequence'].str.len().max()
#print(ending_sequence)
ending_sequence =  27299
starting_sequence = 0

for sequenceCutStart in range(starting_sequence, ending_sequence, sequence_cut_length):

    df['GenomeSequenceSliced'] = df['genome_sequence'].apply(lambda x: x[sequenceCutStart : sequenceCutStart+sequence_cut_length])
    #print(df['GenomeSequenceSliced'])
    print(f"Now working on sequence {sequenceCutStart} to {sequenceCutStart+sequence_cut_length-1}")


    trainEnd = 8
    validationEnd = 9
    n_steps = 1

    train_genome, mutated_genome_train = prepare_data(df['GenomeSequenceSliced'].tolist()[:trainEnd], n_steps)
    validation_genome, mutated_genome_validation = prepare_data(df['GenomeSequenceSliced'].tolist()[trainEnd-1:validationEnd], n_steps)

    mutated_genome_train = ["\t" + genome + "\n" for genome in mutated_genome_train]
    mutated_genome_validation = ["\t" + genome + "\n" for genome in mutated_genome_validation]

    """
    print("Training genomes : ")
    print(f"{train_genome} \n\n")

    print("Mutated genomes Train : ")
    print(f"{mutated_genome_train} \n\n")

    print("Validation genomes : ")
    print(f"{validation_genome} \n\n")

    print("Mutated genomes Validation : ")
    print(f"{mutated_genome_validation} \n\n")

    print("Test genomes : ")
    print(f"{test_genome} \n\n")

    print("Mutated genomes Test : ")
    print(f"{mutated_genome_test} \n\n")
    """

    # Vectorize the data.
    # Train

    input_texts = train_genome
    target_texts = mutated_genome_train
    input_characters = set()
    target_characters = set()

    for input_text in input_texts:
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)

    for target_text in target_texts:
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

    input_characters.add(" ")
    target_characters.add(" ")

    input_characters = sorted(list(input_characters))
    target_characters = sorted(list(target_characters))
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])
    """
    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)
    """
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
        dtype="float32",
    )
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
        dtype="float32",
    )
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
        dtype="float32",
    )

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0



    # Vectorize the data.
    # Validation

    input_texts_validation = validation_genome
    target_texts_validation = mutated_genome_validation
    input_characters_validation = set()
    target_characters_validation = set()

    for input_text_validation in input_texts_validation:
        for char in input_text_validation:
            if char not in input_characters_validation:
                input_characters_validation.add(char)

    for target_text_validation in target_texts_validation:
        for char in target_text_validation:
            if char not in target_characters_validation:
                target_characters_validation.add(char)

    input_characters_validation.add(" ")
    target_characters_validation.add(" ")

    input_characters_validation = sorted(list(input_characters_validation))
    target_characters_validation = sorted(list(target_characters_validation))
    num_encoder_tokens_validation = len(input_characters_validation)
    num_decoder_tokens_validation = len(target_characters_validation)
    max_encoder_seq_length_validation = max([len(txt) for txt in input_texts_validation])
    max_decoder_seq_length_validation = max([len(txt) for txt in target_texts_validation])
    """
    print("Number of samples:", len(input_texts_validation))
    print("Number of unique input tokens:", num_encoder_tokens_validation)
    print("Number of unique output tokens:", num_decoder_tokens_validation)
    print("Max sequence length for inputs:", max_encoder_seq_length_validation)
    print("Max sequence length for outputs:", max_decoder_seq_length_validation)
    """
    input_token_index_validation = dict([(char, i) for i, char in enumerate(input_characters_validation)])
    target_token_index_validation = dict([(char, i) for i, char in enumerate(target_characters_validation)])

    encoder_input_data_validation = np.zeros(
        (len(input_texts_validation), max_encoder_seq_length_validation, num_encoder_tokens_validation),
        dtype="float32",
    )
    decoder_input_data_validation = np.zeros(
        (len(input_texts_validation), max_decoder_seq_length_validation, num_decoder_tokens_validation),
        dtype="float32",
    )
    decoder_target_data_validation = np.zeros(
        (len(input_texts_validation), max_decoder_seq_length_validation, num_decoder_tokens_validation),
        dtype="float32",
    )

    for i, (input_text_validation, target_text_validation) in enumerate(zip(input_texts_validation, target_texts_validation)):
        for t, char in enumerate(input_text_validation):
            encoder_input_data_validation[i, t, input_token_index_validation[char]] = 1.0
        encoder_input_data_validation[i, t + 1 :, input_token_index_validation[" "]] = 1.0
        for t, char in enumerate(target_text_validation):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data_validation[i, t, target_token_index_validation[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data_validation[i, t - 1, target_token_index_validation[char]] = 1.0
        decoder_input_data_validation[i, t + 1 :, target_token_index_validation[" "]] = 1.0
        decoder_target_data_validation[i, t:, target_token_index_validation[" "]] = 1.0


    # Build the model

    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():

        encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
        encoder = keras.layers.LSTM(latent_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)

        encoder_states = [state_h, state_c]

        decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

        decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
        decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
        decoder_outputs = decoder_dense(decoder_outputs)

        model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

        model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

        checkpoint_filepath = '/kaggle/working/s2s_model.h5'
        model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_filepath,
            monitor = 'val_accuracy',
            mode = 'max',
            save_freq = "epoch",
            save_best_only = True,
            verbose=0)

        #model.save("s2s_model.keras")

        #plot_path = 'model_plot.png'
        # Save the model plot to a file
        #plot_model(model, to_file=plot_path, show_shapes=True)
        #model.summary()

        history = model.fit(
            [encoder_input_data, decoder_input_data], decoder_target_data,
            batch_size=batch_size,
            epochs=epochs,
            validation_data = [[encoder_input_data_validation, decoder_input_data_validation], decoder_target_data_validation],
            callbacks = [model_checkpoint_callback]
        )

        # Find the epoch with the highest validation accuracy
        best_epoch = history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1
        best_accuracy = max(history.history['val_accuracy'])

        print(f"\n Best accuracy: {best_accuracy} at epoch {best_epoch}")
        # store the accuracy for this model
        model_name = f'sequence_{sequenceCutStart}_to_{sequenceCutStart+sequence_cut_length-1}'
        best_accuracies[model_name] = best_accuracy

        # Plot training & validation accuracy values
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        title_text = f'Model accuracy for sequence {sequenceCutStart} to {sequenceCutStart+sequence_cut_length-1} \n'
        title_text += f'Best accuracy: {best_accuracy} at epoch {best_epoch}'
        plt.title(title_text)
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(['Train', 'Validation'], loc='upper left')
        accuracy_plot_filename = f'accuracy_plot_{sequenceCutStart}_to_{sequenceCutStart+sequence_cut_length-1}.png'
        plt.savefig(accuracy_plot_filename)  # Save the plot as an image
        plt.show()

        # Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title(f'Model loss for sequence {sequenceCutStart} to {sequenceCutStart+sequence_cut_length-1}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Train', 'Validation'], loc='upper left')
        loss_plot_filename = f'loss_plot_{sequenceCutStart}_to_{sequenceCutStart+sequence_cut_length-1}.png'
        plt.savefig(loss_plot_filename)  # Save the plot as an image
        plt.show()


In [ ]:
# save the best validation accuracies dictionary in the json file

file_path = f'best_validation_accuracies_for_sequence_{starting_sequence}_to_{sequenceCutStart+sequence_cut_length-1}.txt'

# Save the dictionary to a text file
with open(file_path, 'w') as file:
    json.dump(best_accuracies, file)

# print the average of the best validation accuracies
# Calculate the average accuracy
average_accuracy = sum(best_accuracies.values()) / len(best_accuracies)

print(f"The average accuracy for sequence {starting_sequence} to {sequenceCutStart+sequence_cut_length-1} is: {average_accuracy}")